In [134]:
import pandas as pd
from pathlib import Path
import tempfile
import io
from mrich import print
from datetime import datetime

In [135]:
mdf = pd.read_excel("data/InVivo2/JM_InVivo2_EchoRandomisation_max.xlsx", 
                   usecols="B:G", skiprows=1,
    )
# df = df.set_index(["Timepoint", "Animal number", "Rat"])
indices = ["Timepoint", "Treatment", "Rat", "Animal number"]
mdf = mdf.set_index("Randomised number")
mdf.loc[mdf["Rat"] == "NM", "Rat"] = 79
mdf["Treatment"] = mdf["Rat"].apply(lambda x: "Control" if x%4<2 else "IR")
mdf

,Timepoint,Animal number,Rat,Ear Notch,Acquisition Date,Treatment
Randomised number,,,,,,
53,Baseline,M00681267,64,L,2024-08-31,Control
47,Baseline,M00681268,65,R,2024-08-31,Control
128,Baseline,M00681269,66,LL,2024-08-31,IR
52,Baseline,M00681270,67,NM,2024-09-02,IR
40,Baseline,M00681271,68,L,2024-08-31,Control
...,...,...,...,...,...,...
74,25 wk,M00681278,75,NM,2025-03-28,IR
57,25 wk,M00681279,76,L,2025-03-28,Control
28,25 wk,M00681280,77,R,2025-03-28,Control


In [136]:
data = {}

### PSLAX Bmode

keys = [
    "PSLAX_Bmode",
    "PSLAX_Mmode",
    "PSSAX_Mmode",
]

for key in keys:
    for file in Path("data/InVivo2/blinded_echo").glob(f"202?????_InVivo2_Blinded_*_{key}.csv"):
        
        date, _, _, image_num, _, _ = file.name.split("_")
        
        image_num = int(image_num)
        
        match key:
            case "PSLAX_Bmode":
                search = '"Measurement","Mode","Parameter","Units",'
                n_lines = 11
            case "PSLAX_Mmode" | "PSSAX_Mmode":
                search = '"Measurement","Mode","Parameter","Units","Avg","STD","Instance 1","Instance 2","Instance 3"'
                n_lines = 6
            case _:
                raise ValueError

        cmds = [
            "grep",
            f"-A{n_lines}",
            search,
            file,
        ]

        result = subprocess.run(cmds, stdout=subprocess.PIPE, text=True)
        df = pd.read_csv(io.StringIO(result.stdout), index_col=False)

        d = data.setdefault(image_num, {})
        
        d.update({
            "Randomised number":int(image_num),
            f"{key}: Date": datetime.strptime(date, "%Y%m%d").date(),
        })

        match key:
            case "PSLAX_Bmode":
                for i,row in df.iterrows():
                    param = row["Parameter"]
                    unit = row["Units"]
                    value = row[-1]
                    d[f"{key}: {param} [{unit}]"] = value
            case "PSLAX_Mmode" | "PSSAX_Mmode":
                for i,row in df.iterrows():
                    param = row["Measurement"]
                    unit = row["Units"]
                    d[f"{key}: {param} Avg [{unit}]"] = row["Avg"]
                    d[f"{key}: {param} STD [{unit}]"] = row["STD"]

#### COMBINE

df = pd.DataFrame(data.values())
# df = df.set_index("Randomised number")

#### MAP TO RANDOM NUMBERS

for col in indices:
    df[col] = df["Randomised number"].apply(lambda x: mdf.loc[x,col])

#### SORT

timepoints = ["Baseline", "4 wk", "8 wk", "10 wk", "14 wk", "18 wk", "22 wk", "25 wk"]

df["Timepoint"] = pd.Categorical(df["Timepoint"], categories=timepoints, ordered=True)
    
df = df.sort_values(by=indices)
df = df.set_index(indices + ["Randomised number"])

df

PSLAX_Bmode: Date  \
Timepoint Treatment Rat Animal number Randomised number                     
Baseline  Control   64  M00681267     53                       2025-04-28   
                    65  M00681268     47                       2025-04-26   
                    68  M00681271     40                       2025-04-26   
                    69  M00681272     117                      2025-05-02   
                    72  M00681275     76                       2025-04-28   
...                                                                   ...   
25 wk     IR        71  M00681274     18                       2025-04-25   
                    74  M00681277     65                       2025-04-28   
                    75  M00681278     74                       2025-04-28   
                    78  M00681281     21                       2025-04-25   
                    79  M00681282     124                      2025-05-02   

                                                         PSLAX_Bmode: Heart Rate [BPM]  \
Timepoint Treatment Rat Animal number Randomised number                                  
Baseline  Control   64  M00681267     53                                    428.189117   
                    65  M00681268     47                                    435.808970   
                    68  M00681271     40                                    468.566966   
                    69  M00681272     117                                   418.628990   
                    72  M00681275     76                                    399.600400   
...                                                                                ...   
25 wk     IR        71  M00681274     18                                    345.622120   
                    74  M00681277     65                                    352.164343   
                    75  M00681278     74                                    370.084811   
                    78  M00681281     21                                    351.699883   
                    79  M00681282     124                                   361.391357   

                                                         PSLAX_Bmode: Area [mm2]  \
Timepoint Treatment Rat Animal number Randomised number                            
Baseline  Control   64  M00681267     53                               30.421540   
                    65  M00681268     47                               32.815279   
                    68  M00681271     40                               28.106887   
                    69  M00681272     117                              34.847759   
                    72  M00681275     76                               29.239489   
...                                                                          ...   
25 wk     IR        71  M00681274     18                               35.601831   
                    74  M00681277     65                               70.887392   
                    75  M00681278     74                               38.012953   
                    78  M00681281     21                               25.901006   
                    79  M00681282     124                              40.650569   

                                                         PSLAX_Bmode: Area;s [mm2]  \
Timepoint Treatment Rat Animal number Randomised number                              
Baseline  Control   64  M00681267     53                                 26.736552   
                    65  M00681268     47                                 30.244550   
                    68  M00681271     40                                 26.581895   
                    69  M00681272     117                                33.943376   
                    72  M00681275     76                                 27.919859   
...                                                                            ...   
25 wk     IR        71  M00681274     18                                 32.689276   
                

In [137]:
df.to_excel("InVivo2_echo_processed.xlsx")